In [1]:
import pandas as pd
import glob
import codecs
from pandas import Series, DataFrame

from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout
from keras.optimizers import Adam,RMSprop,SGD
from keras.utils import np_utils
from keras import optimizers,regularizers
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

#最大表示列数の指定（ここでは50列を指定）
pd.set_option('display.max_columns', 100)
#最大表示行数の指定（ここでは50行を指定）
pd.set_option('display.max_rows', 100)

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#C50

#圧力_温度_配管ID.csvの読み込み
data1 = pd.read_csv("drive/My Drive/mdb/test/C50/圧力_温度_配管ID.csv", encoding='shift_jis')
#配管特性.csvの読み込み
data2 = pd.read_csv('drive/My Drive/mdb/test/C50/配管特性.csv',encoding='shift_jis')
#圧力_温度_配管ID.csvのデータの配管IDに合わせて配管特性.csvのデータを結合
data1_frame = DataFrame(data1)
data2_frame = DataFrame(data2)
data12 = pd.merge(data1_frame,data2_frame,how='left',left_on='配管 ID',right_on = '配管ID')
#'To'データの削除
data12 = data12.drop("To",axis=1)
for col_name in data12.columns:
    print(col_name,data12[col_name].dtype)
#'配管ID'などの文字データのダミー化
data12_dummy = pd.get_dummies(data12[data12.columns[data12.columns != 'From']])
data12_dummy['From'] = data12['From']
#Support.csvの読み込み
data_out = pd.read_csv("drive/My Drive/mdb/test/C50/Support.csv",encoding='shift_jis')
#使用したい "Point","Type"データのみにする
for col_name in data_out.columns:
    if col_name != "Point" and col_name != "Type" :
        data_out = data_out.drop(col_name,axis=1)
#重複しているデータの削除
data_out = data_out.drop_duplicates()
#入力データと出力データを'Point'データで結合
data_press_type = pd.merge(data12_dummy,data_out,how = "outer",left_on = 'From',right_on='Point')
#文字データなのでダミー化
data_press_type.at[data_press_type['Type'] == "Guide",'Type'] = 1
data_press_type.at[data_press_type['Type'] == "V - Stop",'Type'] = 2
data_press_type.at[data_press_type['Type'] == "Spr. Hgr",'Type'] = 3
data_press_type.at[data_press_type['Type'] == "Spr. Can",'Type'] = 4
data_test_press = data_press_type.fillna(0)
data_series = Series(data_test_press['Type'])
print(pd.Series.value_counts(data_series))
data_test_press = data_test_press[data_test_press["呼び径 (mm)_x"] != 0]
data_test_press

From object
材質 object
配管 ID object
呼び径 (mm)_x int64
ｹｰｽ 1 圧力 (N/mm2) float64
ｹｰｽ 1 温度 (deg C) int64
ｹｰｽ 1 自動_1 int64
ｹｰｽ 1 線膨張量 (mm/m) float64
ｹｰｽ 1 自動_2 int64
ｹｰｽ 1 運転時弾性係数 (E6 N/mm2) float64
ｹｰｽ 1 自動_3 int64
ｹｰｽ 1 運転時許容応力 (N/mm2) float64
配管ID object
ﾀｸﾞ ﾅﾝﾊﾞｰ object
呼び径 (mm)_y int64
実外径 (mm) float64
ｽｹｼﾞｭｰﾙ object
肉厚 (mm) float64
腐れ代 (mm) int64
製作公差 (mm) float64
保温厚さ (mm) int64
保温材質 object
保温材密度 (kg/m3) float64
外装板厚 (mm) int64
外装板材質 object
外装板密度 (kg/m3) object
ﾗｲﾆﾝｸﾞ厚さ (mm) int64
ﾗｲﾆﾝｸﾞ密度 (kg/m3) object
ﾗｲﾝ ｸﾗｽ object
流体比重 int64
配管材質 object
最低温度の警告 int64
溶接E係数 int64
溶接W係数 自動 object
溶接W係数 object
応力振幅 低減係数 int64
常温時許容応力 (N/mm2) float64
最小降伏点(Sy) (N/mm2) int64
引張強さ(Su) (N/mm2) int64
縦弾性係数 (E6 N/mm2) float64
周方向弾性係数 (E6 N/mm2) float64
せん断弾性係数 (E6 N/mm2) float64
密度 (kg/m3) int64
ﾎﾟｱｿﾝ比 float64
0    277
1     35
2     17
3     12
4      1
Name: Type, dtype: int64


,呼び径 (mm)_x,ｹｰｽ 1 圧力 (N/mm2),ｹｰｽ 1 温度 (deg C),ｹｰｽ 1 自動_1,ｹｰｽ 1 線膨張量 (mm/m),ｹｰｽ 1 自動_2,ｹｰｽ 1 運転時弾性係数 (E6 N/mm2),ｹｰｽ 1 自動_3,ｹｰｽ 1 運転時許容応力 (N/mm2),呼び径 (mm)_y,実外径 (mm),肉厚 (mm),腐れ代 (mm),製作公差 (mm),保温厚さ (mm),保温材密度 (kg/m3),外装板厚 (mm),ﾗｲﾆﾝｸﾞ厚さ (mm),流体比重,最低温度の警告,溶接E係数,応力振幅 低減係数,常温時許容応力 (N/mm2),最小降伏点(Sy) (N/mm2),引張強さ(Su) (N/mm2),縦弾性係数 (E6 N/mm2),周方向弾性係数 (E6 N/mm2),せん断弾性係数 (E6 N/mm2),密度 (kg/m3),ﾎﾟｱｿﾝ比,材質_SB410,材質_STPT370-S,配管 ID_E350S40,配管 ID_E400T13,配管 ID_E500T13,配管 ID_P100S40,配管 ID_P125S40,配管 ID_P150S40,配管 ID_P200S40,配管 ID_P250S40,配管 ID_P350S40,配管 ID_P450S40,配管 ID_P450STD,配管 ID_P450T13,配管 ID_P550STD,配管 ID_P600STD,配管 ID_P600T13,配管 ID_P700STD,配管ID_E350S40,配管ID_E400T13,配管ID_E500T13,配管ID_P100S40,配管ID_P125S40,配管ID_P150S40,配管ID_P200S40,配管ID_P250S40,配管ID_P350S40,配管ID_P450S40,配管ID_P450STD,配管ID_P450T13,配管ID_P550STD,配管ID_P600STD,配管ID_P600T13,配管ID_P700STD,ﾀｸﾞ ﾅﾝﾊﾞｰ_,ｽｹｼﾞｭｰﾙ_40,ｽｹｼﾞｭｰﾙ_NS,ｽｹｼﾞｭｰﾙ_STD,保温材質_Other,外装板材質_,外装板密度 (kg/m3)_,ﾗｲﾆﾝｸﾞ密度 (kg/m3)_,ﾗｲﾝ ｸﾗｽ_BC22,ﾗｲﾝ ｸﾗｽ_BC23,ﾗｲﾝ ｸﾗｽ_BC24,配管材質_SB410,配管材質_STPT370-S,溶接W係数 自動_,溶接W係数_,From,Point,Type
0,250.0,1.77,375.0,1.0,4.750695,1.0,0.17450,1.0,89.00,250.0,267.4,9.3,0.0,1.1625,65.0,201.47,0.0,0.0,0.0,0.0,1.0,1.0,92.0,215.0,370.0,0.202927,0.202927,0.078049,7850.0,0.3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,A00,0,0
1,250.0,1.77,375.0,1.0,4.750695,1.0,0.17450,1.0,89.00,250.0,267.4,9.3,0.0,1.1625,65.0,201.47,0.0,0.0,0.0,0.0,1.0,1.0,92.0,215.0,370.0,0.202927,0.202927,0.078049,7850.0,0.3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,A01,0,0
2,250.0,1.77,375.0,1.0,4.750695,1.0,0.17450,1.0,89.00,250.0,267.4,9.3,0.0,1.1625,65.0,201.47,0.0,0.0,0.0,0.0,1.0,1.0,92.0,215.0,370.0,0.202927,0.202927,0.078049,7850.0,0.3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,A02 N,0,0
3,250.0,1.77,375.0,1.0,4.750695,1.0,0.17450,1.0,89.00,250.0,267.4,9.3,0.0,1.1625,65.0,201.47,0.0,0.0,0.0,0.0,1.0,1.0,92.0,215.0,370.0,0.202927,0.202927,0.078049,7850.0,0.3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,A02 F,0,0
4,250.0,1.77,375.0,1.0,4.750695,1.0,0.17450,1.0,89.00,250.0,267.4,9.3,0.0,1.1625,65.0,201.47,0.0,0.0,0.0,0.0,1.0,1.0,92.0,215.0,370.0,0.202927,0.202927,0.078049,7850.0,0.3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,A03 N,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,100.0,1.77,375.0,1.0,4.750695,1.0,0.17450,1.0,89.00,100.0,114.3,6.0,0.0,0.7500,65.0,201.53,0.0,0.0,0.0,0.0,1.0,1.0,92.0,215.0,370.0,0.202927,0.202927,0.078049,7850.0,0.3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,I01 F,0,0
334,100.0,1.77,375.0,1.0,4.750695,1.0,0.17450,1.0,89.00,100.0,114.3,6.0,0.0,0.7500,65.0,201.53,0.0,0.0,0.0,0.0,1.0,1.0,92.0,215.0,370.0,0.202927,0.202927,0.078049,7850.0,0.3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,

In [0]:
np.random.seed(0) # 乱数を固定値で初期化し再現性を持たせる
X = data_test_press.drop(['From','Point','Type'],axis=1)
T = data_test_press['Type']

X_train,X_test, y_train, y_test = train_test_split(X, T, train_size=0.7, test_size=0.3)

In [0]:
from sklearn.metrics import confusion_matrix,accuracy_score
 
#決定木
from sklearn.tree import DecisionTreeClassifier
 
dt = DecisionTreeClassifier(max_depth=3,random_state=0)
dt.fit(X_train,y_train)
y_pred = dt.predict(X_test)
print("dt_accuracy_score : {}".format(accuracy_score(y_test,y_pred)))
 
 
#ランダムフォレスト
from sklearn.ensemble import RandomForestClassifier
 
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
print("rf_accuracy_score : {}".format(accuracy_score(y_test,y_pred)))
 
#XGBoost
import xgboost as xgb
from xgboost import XGBClassifier
 
xgb = xgb.XGBClassifier(booster="gbtree",silent=0,seed=0)
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
print("xgb_accuracy_score : {}".format(accuracy_score(y_test,y_pred)))

dt_accuracy_score : 0.8333333333333334
rf_accuracy_score : 0.8333333333333334
xgb_accuracy_score : 0.8333333333333334


In [0]:
#3つのモデルの特徴量重要度を並べてみる
models = ["DT","RF","XGB"]
importances = pd.DataFrame({"features":X_train.columns.values,
                            models[0]:dt.feature_importances_,
                            models[1]:rf.feature_importances_,
                            models[2]:xgb.feature_importances_})
 
#データフレームをDTの降順に並び替えて表示
importances.sort_values("DT",ascending=False)

,features,DT,RF,XGB
8,ｹｰｽ 1 運転時許容応力 (N/mm2),0.493947,0.049227,0.188056
36,配管 ID_P125S40,0.247506,0.043328,0.106279
55,配管ID_P250S40,0.135059,0.026237,0.000000
43,配管 ID_P450T13,0.087223,0.018480,0.044547
46,配管 ID_P600T13,0.036264,0.013237,0.130602
0,呼び径 (mm)_x,0.000000,0.060813,0.092038
51,配管ID_P100S40,0.000000,0.005517,0.000000
57,配管ID_P450S40,0.000000,0.002003,0.000000
56,配管ID_P350S40,0.000000,0.001037,0.000000
54,配管ID_P200S40,0.000000,0.000636,0.000000


In [0]:
#C61

#圧力_温度_配管ID.csvの読み込み
data1 = pd.read_csv("drive/My Drive/mdb/test/C61/圧力_温度_配管ID.csv", encoding='shift_jis')
#配管特性.csvの読み込み
data2 = pd.read_csv('drive/My Drive/mdb/test/C61/配管特性.csv',encoding='shift_jis')
#圧力_温度_配管ID.csvのデータの配管IDに合わせて配管特性.csvのデータを結合
data1_frame = DataFrame(data1)
data2_frame = DataFrame(data2)
data12 = pd.merge(data1_frame,data2_frame,how='left',left_on='配管 ID',right_on = '配管ID')
#'To'データの削除
data12 = data12.drop("To",axis=1)
for col_name in data12.columns:
    print(col_name,data12[col_name].dtype)
#'配管ID'などの文字データのダミー化
data12_dummy = pd.get_dummies(data12[data12.columns[data12.columns != 'From']])
data12_dummy['From'] = data12['From']
#Support.csvの読み込み
data_out = pd.read_csv("drive/My Drive/mdb/test/C61/Support.csv",encoding='shift_jis')
#使用したい "Point","Type"データのみにする
for col_name in data_out.columns:
    if col_name != "Point" and col_name != "Type" :
        data_out = data_out.drop(col_name,axis=1)
#重複しているデータの削除
data_out = data_out.drop_duplicates()
#入力データと出力データを'Point'データで結合
data_press_type = pd.merge(data12_dummy,data_out,how = "outer",left_on = 'From',right_on='Point')
#文字データなのでダミー化
data_press_type.at[data_press_type['Type'] == "Guide",'Type'] = 1
data_press_type.at[data_press_type['Type'] == "V - Stop",'Type'] = 2
data_press_type.at[data_press_type['Type'] == "Spr. Hgr",'Type'] = 3
data_press_type.at[data_press_type['Type'] == "Inclined",'Type'] = 4
data_test_press = data_press_type.fillna(0)
data_series = Series(data_test_press['Type'])
print(pd.Series.value_counts(data_series))
data_test_press = data_test_press[data_test_press["呼び径 (mm)_x"] != 0]
data_test_press

From object
材質 object
配管 ID object
呼び径 (mm)_x float64
ｹｰｽ 1 圧力 (N/mm2) float64
ｹｰｽ 1 温度 (deg C) float64
ｹｰｽ 1 自動_1 float64
ｹｰｽ 1 線膨張量 (mm/m) float64
ｹｰｽ 1 自動_2 float64
ｹｰｽ 1 運転時弾性係数 (E6 N/mm2) float64
ｹｰｽ 1 自動_3 float64
ｹｰｽ 1 運転時許容応力 (N/mm2) float64
配管ID object
ﾀｸﾞ ﾅﾝﾊﾞｰ object
呼び径 (mm)_y int64
実外径 (mm) float64
ｽｹｼﾞｭｰﾙ int64
肉厚 (mm) float64
腐れ代 (mm) float64
製作公差 (mm) float64
保温厚さ (mm) float64
保温材質 object
保温材密度 (kg/m3) float64
外装板厚 (mm) float64
外装板材質 object
外装板密度 (kg/m3) object
ﾗｲﾆﾝｸﾞ厚さ (mm) float64
ﾗｲﾆﾝｸﾞ密度 (kg/m3) object
ﾗｲﾝ ｸﾗｽ object
流体比重 float64
配管材質 object
最低温度の警告 float64
溶接E係数 float64
溶接W係数 自動 object
溶接W係数 object
応力振幅 低減係数 float64
常温時許容応力 (N/mm2) float64
最小降伏点(Sy) (N/mm2) float64
引張強さ(Su) (N/mm2) float64
縦弾性係数 (E6 N/mm2) float64
周方向弾性係数 (E6 N/mm2) float64
せん断弾性係数 (E6 N/mm2) float64
密度 (kg/m3) float64
ﾎﾟｱｿﾝ比 float64
0    60
2     4
1     3
3     2
4     1
Name: Type, dtype: int64


,呼び径 (mm)_x,ｹｰｽ 1 圧力 (N/mm2),ｹｰｽ 1 温度 (deg C),ｹｰｽ 1 自動_1,ｹｰｽ 1 線膨張量 (mm/m),ｹｰｽ 1 自動_2,ｹｰｽ 1 運転時弾性係数 (E6 N/mm2),ｹｰｽ 1 自動_3,ｹｰｽ 1 運転時許容応力 (N/mm2),呼び径 (mm)_y,実外径 (mm),ｽｹｼﾞｭｰﾙ,肉厚 (mm),腐れ代 (mm),製作公差 (mm),保温厚さ (mm),保温材密度 (kg/m3),外装板厚 (mm),ﾗｲﾆﾝｸﾞ厚さ (mm),流体比重,最低温度の警告,溶接E係数,応力振幅 低減係数,常温時許容応力 (N/mm2),最小降伏点(Sy) (N/mm2),引張強さ(Su) (N/mm2),縦弾性係数 (E6 N/mm2),周方向弾性係数 (E6 N/mm2),せん断弾性係数 (E6 N/mm2),密度 (kg/m3),ﾎﾟｱｿﾝ比,材質_STPA12-S,配管 ID_E300S120,配管 ID_P250S120,配管 ID_P350S120,配管ID_E300S120,配管ID_P250S120,配管ID_P350S120,ﾀｸﾞ ﾅﾝﾊﾞｰ_,保温材質_Other,外装板材質_,外装板密度 (kg/m3)_,ﾗｲﾆﾝｸﾞ密度 (kg/m3)_,ﾗｲﾝ ｸﾗｽ_EH21,配管材質_STPA12-S,溶接W係数 自動_,溶接W係数_,From,Point,Type
0,300.0,10.6,487.0,1.0,6.58,1.0,0.15,1.0,83.51,300,318.5,120,25.4,0.0,3.17,150.0,201.23,0.0,0.0,0.0,0.0,1.0,1.0,95.0,205.0,380.0,0.19,0.19,0.07,7850.0,0.3,1,1,0,0,1,0,0,1,1,1,1,1,1,1,1,1,A00,0,0
1,350.0,10.6,487.0,1.0,6.58,1.0,0.15,1.0,83.51,350,355.6,120,27.8,0.0,3.47,150.0,201.23,0.0,0.0,0.0,0.0,1.0,1.0,95.0,205.0,380.0,0.19,0.19,0.07,7850.0,0.3,1,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,A01,0,0
2,350.0,10.6,487.0,1.0,6.58,1.0,0.15,1.0,83.51,350,355.6,120,27.8,0.0,3.47,150.0,201.23,0.0,0.0,0.0,0.0,1.0,1.0,95.0,205.0,380.0,0.19,0.19,0.07,7850.0,0.3,1,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,A02 N,0,0
3,350.0,10.6,487.0,1.0,6.58,1.0,0.15,1.0,83.51,350,355.6,120,27.8,0.0,3.47,150.0,201.23,0.0,0.0,0.0,0.0,1.0,1.0,95.0,205.0,380.0,0.19,0.19,0.07,7850.0,0.3,1,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,A02 F,0,0
4,350.0,10.6,487.0,1.0,6.58,1.0,0.15,1.0,83.51,350,355.6,120,27.8,0.0,3.47,150.0,201.23,0.0,0.0,0.0,0.0,1.0,1.0,95.0,205.0,380.0,0.19,0.19,0.07,7850.0,0.3,1,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,A03 N,0,0
5,350.0,10.6,487.0,1.0,6.58,1.0,0.15,1.0,83.51,350,355.6,120,27.8,0.0,3.47,150.0,201.23,0.0,0.0,0.0,0.0,1.0,1.0,95.0,205.0,380.0,0.19,0.19,0.07,7850.0,0.3,1,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,A03 F,0,0
6,350.0,10.6,487.0,1.0,6.58,1.0,0.15,1.0,83.51,350,355.6,120,27.8,0.0,3.47,150.0,201.23,0.0,0.0,0.0,0.0,1.0,1.0,95.0,205.0,380.0,0.19,0.19,0.07,7850.0,0.3,1,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,A04,0,0
7,350.0,10.6,487.0,1.0,6.58,1.0,0.15,1.0,83.51,350,355.6,120,27.8,0.0,3.47,150.0,201.23,0.0,0.0,0.0,0.0,1.0,1.0,95.0,205.0,380.0,0.19,0.19,0.07,7850.0,0.3,1,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,A23,A23,3
8,350.0,10.6,487.0,1.0,6.58,1.0,0.15,1.0,83.51,350,355.6,120,27.8,0.0,3.47,150.0,201.23,0.0,0.0,0.0,0.0,1.0,1.0,95.0,205.0,380.0,0.19,0.19,0.07,7850.0,0.3,1,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,A05 N,0,0
9,350.0,10.6,487.0,1.0,6.58,1.0,0.15,1.0,83.51,350,355.6,120,27.8,0.0,3.47,150.0,201.23,0.0,0.0,0.0,0.0,1.0,1.0,95.0,205.0,380.0,0.19,0.19,0.07,7850.0,0.3,1,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,A05 F,0,0


In [0]:
np.random.seed(0) # 乱数を固定値で初期化し再現性を持たせる
X = data_test_press.drop(['From','Point','Type'],axis=1)
T = data_test_press['Type']

X_train,X_test, y_train, y_test = train_test_split(X, T, train_size=0.7, test_size=0.3)

In [0]:
from sklearn.metrics import confusion_matrix,accuracy_score
 
#決定木
from sklearn.tree import DecisionTreeClassifier
 
dt = DecisionTreeClassifier(max_depth=3,random_state=0)
dt.fit(X_train,y_train)
y_pred = dt.predict(X_test)
print("dt_accuracy_score : {}".format(accuracy_score(y_test,y_pred)))
 
 
#ランダムフォレスト
from sklearn.ensemble import RandomForestClassifier
 
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
print("rf_accuracy_score : {}".format(accuracy_score(y_test,y_pred)))
 
#XGBoost
import xgboost as xgb
from xgboost import XGBClassifier
 
xgb = xgb.XGBClassifier(booster="gbtree",silent=0,seed=0)
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
print("xgb_accuracy_score : {}".format(accuracy_score(y_test,y_pred)))

#3つのモデルの特徴量重要度を並べてみる
models = ["DT","RF","XGB"]
importances = pd.DataFrame({"features":X_train.columns.values,
                            models[0]:dt.feature_importances_,
                            models[1]:rf.feature_importances_,
                            models[2]:xgb.feature_importances_})
 
#データフレームをDTの降順に並び替えて表示
importances.sort_values("DT",ascending=False)

dt_accuracy_score : 0.8095238095238095
rf_accuracy_score : 0.8095238095238095
xgb_accuracy_score : 0.8095238095238095


,features,DT,RF,XGB
34,配管 ID_P350S120,1.0,0.071225,0.0
0,呼び径 (mm)_x,0.0,0.099570,1.0
35,配管ID_E300S120,0.0,0.005396,0.0
26,縦弾性係数 (E6 N/mm2),0.0,0.000000,0.0
27,周方向弾性係数 (E6 N/mm2),0.0,0.000000,0.0
28,せん断弾性係数 (E6 N/mm2),0.0,0.000000,0.0
29,密度 (kg/m3),0.0,0.000000,0.0
30,ﾎﾟｱｿﾝ比,0.0,0.000000,0.0
31,材質_STPA12-S,0.0,0.000000,0.0
32,配管 ID_E300S120,0.0,0.005245,0.0


In [0]:
#L41

#圧力_温度_配管ID.csvの読み込み
data1 = pd.read_csv("drive/My Drive/mdb/test/L41/圧力_温度_配管ID.csv", encoding='shift_jis')
#配管特性.csvの読み込み
data2 = pd.read_csv('drive/My Drive/mdb/test/L41/配管特性.csv',encoding='shift_jis')
#圧力_温度_配管ID.csvのデータの配管IDに合わせて配管特性.csvのデータを結合
data1_frame = DataFrame(data1)
data2_frame = DataFrame(data2)
data12 = pd.merge(data1_frame,data2_frame,how='left',left_on='配管 ID',right_on = '配管ID')
#'To'データの削除
data12 = data12.drop("To",axis=1)
for col_name in data12.columns:
    print(col_name,data12[col_name].dtype)
#'配管ID'などの文字データのダミー化
data12_dummy = pd.get_dummies(data12[data12.columns[data12.columns != 'From']])
data12_dummy['From'] = data12['From']
#Support.csvの読み込み
data_out = pd.read_csv("drive/My Drive/mdb/test/L41/Support.csv",encoding='shift_jis')
#使用したい "Point","Type"データのみにする
for col_name in data_out.columns:
    if col_name != "Point" and col_name != "Type" :
        data_out = data_out.drop(col_name,axis=1)
#重複しているデータの削除
data_out = data_out.drop_duplicates()
#入力データと出力データを'Point'データで結合
data_press_type = pd.merge(data12_dummy,data_out,how = "outer",left_on = 'From',right_on='Point')
#文字データなのでダミー化
data_press_type.at[data_press_type['Type'] == "Guide",'Type'] = 1
data_press_type.at[data_press_type['Type'] == "V - Stop",'Type'] = 2
data_press_type.at[data_press_type['Type'] == "Spr. Hgr",'Type'] = 3
data_press_type.at[data_press_type['Type'] == "Inclined",'Type'] = 4
data_test_press = data_press_type.fillna(0)
data_series = Series(data_test_press['Type'])
print(pd.Series.value_counts(data_series))
data_test_press = data_test_press[data_test_press["呼び径 (mm)_x"] != 0]
data_test_press

From object
材質 object
配管 ID object
呼び径 (mm)_x float64
ｹｰｽ 1 圧力 (N/mm2) float64
ｹｰｽ 1 温度 (deg C) float64
ｹｰｽ 1 自動_1 float64
ｹｰｽ 1 線膨張量 (mm/m) float64
ｹｰｽ 1 自動_2 float64
ｹｰｽ 1 運転時弾性係数 (E6 N/mm2) float64
ｹｰｽ 1 自動_3 float64
ｹｰｽ 1 運転時許容応力 (N/mm2) float64
配管ID object
ﾀｸﾞ ﾅﾝﾊﾞｰ object
呼び径 (mm)_y int64
実外径 (mm) float64
ｽｹｼﾞｭｰﾙ int64
肉厚 (mm) float64
腐れ代 (mm) float64
製作公差 (mm) float64
保温厚さ (mm) float64
保温材質 object
保温材密度 (kg/m3) float64
外装板厚 (mm) float64
外装板材質 object
外装板密度 (kg/m3) object
ﾗｲﾆﾝｸﾞ厚さ (mm) float64
ﾗｲﾆﾝｸﾞ密度 (kg/m3) object
ﾗｲﾝ ｸﾗｽ object
流体比重 float64
配管材質 object
最低温度の警告 float64
溶接E係数 float64
溶接W係数 自動 object
溶接W係数 object
応力振幅 低減係数 float64
常温時許容応力 (N/mm2) float64
最小降伏点(Sy) (N/mm2) float64
引張強さ(Su) (N/mm2) float64
縦弾性係数 (E6 N/mm2) float64
周方向弾性係数 (E6 N/mm2) float64
せん断弾性係数 (E6 N/mm2) float64
密度 (kg/m3) float64
ﾎﾟｱｿﾝ比 float64
0    89
1    14
2     3
4     1
3     1
Name: Type, dtype: int64


,呼び径 (mm)_x,ｹｰｽ 1 圧力 (N/mm2),ｹｰｽ 1 温度 (deg C),ｹｰｽ 1 自動_1,ｹｰｽ 1 線膨張量 (mm/m),ｹｰｽ 1 自動_2,ｹｰｽ 1 運転時弾性係数 (E6 N/mm2),ｹｰｽ 1 自動_3,ｹｰｽ 1 運転時許容応力 (N/mm2),呼び径 (mm)_y,実外径 (mm),ｽｹｼﾞｭｰﾙ,肉厚 (mm),腐れ代 (mm),製作公差 (mm),保温厚さ (mm),保温材密度 (kg/m3),外装板厚 (mm),ﾗｲﾆﾝｸﾞ厚さ (mm),流体比重,最低温度の警告,溶接E係数,応力振幅 低減係数,常温時許容応力 (N/mm2),最小降伏点(Sy) (N/mm2),引張強さ(Su) (N/mm2),縦弾性係数 (E6 N/mm2),周方向弾性係数 (E6 N/mm2),せん断弾性係数 (E6 N/mm2),密度 (kg/m3),ﾎﾟｱｿﾝ比,材質_STPG370-S,配管 ID_P300S40,配管 ID_P350S40,配管 ID_P400S40,配管 ID_P450S40,配管ID_P300S40,配管ID_P350S40,配管ID_P400S40,配管ID_P450S40,ﾀｸﾞ ﾅﾝﾊﾞｰ_,保温材質_Other,外装板材質_,外装板密度 (kg/m3)_,ﾗｲﾆﾝｸﾞ密度 (kg/m3)_,ﾗｲﾝ ｸﾗｽ_BB21,配管材質_STPG370-S,溶接W係数 自動_,溶接W係数_,From,Point,Type
0,400.0,1.1,176.0,1.0,1.87,1.0,0.19,1.0,106.0,400,406.4,40,12.7,0.0,1.58,50.0,201.35,0.0,0.0,1.0,0.0,1.0,1.0,91.99,215.0,370.0,0.2,0.2,0.07,7850.0,0.3,1,0,0,1,0,0,0,1,0,1,1,1,1,1,1,1,1,1,A00,0,0
1,400.0,1.1,176.0,1.0,1.87,1.0,0.19,1.0,106.0,400,406.4,40,12.7,0.0,1.58,50.0,201.35,0.0,0.0,1.0,0.0,1.0,1.0,91.99,215.0,370.0,0.2,0.2,0.07,7850.0,0.3,1,0,0,1,0,0,0,1,0,1,1,1,1,1,1,1,1,1,A01 N,0,0
2,400.0,1.1,176.0,1.0,1.87,1.0,0.19,1.0,106.0,400,406.4,40,12.7,0.0,1.58,50.0,201.35,0.0,0.0,1.0,0.0,1.0,1.0,91.99,215.0,370.0,0.2,0.2,0.07,7850.0,0.3,1,0,0,1,0,0,0,1,0,1,1,1,1,1,1,1,1,1,A01 F,0,0
3,400.0,1.1,176.0,1.0,1.87,1.0,0.19,1.0,106.0,400,406.4,40,12.7,0.0,1.58,50.0,201.35,0.0,0.0,1.0,0.0,1.0,1.0,91.99,215.0,370.0,0.2,0.2,0.07,7850.0,0.3,1,0,0,1,0,0,0,1,0,1,1,1,1,1,1,1,1,1,A02,0,0
4,400.0,1.1,176.0,1.0,1.87,1.0,0.19,1.0,106.0,400,406.4,40,12.7,0.0,1.58,50.0,201.35,0.0,0.0,1.0,0.0,1.0,1.0,91.99,215.0,370.0,0.2,0.2,0.07,7850.0,0.3,1,0,0,1,0,0,0,1,0,1,1,1,1,1,1,1,1,1,A03 N,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,400.0,1.1,176.0,1.0,1.87,1.0,0.19,1.0,106.0,400,406.4,40,12.7,0.0,1.58,50.0,201.35,0.0,0.0,1.0,0.0,1.0,1.0,91.99,215.0,370.0,0.2,0.2,0.07,7850.0,0.3,1,0,0,1,0,0,0,1,0,1,1,1,1,1,1,1,1,1,F01,0,0
104,350.0,1.1,176.0,1.0,1.87,1.0,0.19,1.0,106.0,350,355.6,40,11.1,0.0,1.38,50.0,201.50,0.0,0.0,1.0,0.0,1.0,1.0,91.99,215.0,370.0,0.2,0.2,0.07,7850.0,0.3,1,0,1,0,0,0,1,0,0,1,1,1,1,1,1,1,1,1,F02,0,0
105,350.0,1.1,176.0,1.0,1.87,1.0,0.19,1.0,106.0,350,355.6,40,11.1,0.0,1.38,50.0,201.50,0.0,0.0,1.0,0.0,1.0,1.0,91.99,215.0,370.0,0.2,0.2,0.07,7850.0,0.3,1,0,1,0,0,0,1,0,0,1,1,1,1,1,1,1,1,1,F03,0,0
106,350.0,1.1,176.0,1.0,1.87,1.0,0.19,1.0,106.0,350,355.6,40,11.1,0.0,1.38,50.0,201.50,0.0,0.0,1.0,0.0,1.0,1.0,91.99,215.0,370.0,0.2,0.2,0.07,7850.0,0.3,1,0,1,0,0,0,1,0,0,1,1,1,1,1,1,1,1,1,F04,0,0


In [0]:
np.random.seed(0) # 乱数を固定値で初期化し再現性を持たせる
X = data_test_press.drop(['From','Point','Type'],axis=1)
T = data_test_press['Type']

X_train,X_test, y_train, y_test = train_test_split(X, T, train_size=0.7, test_size=0.3)

from sklearn.metrics import confusion_matrix,accuracy_score
 
#決定木
from sklearn.tree import DecisionTreeClassifier
 
dt = DecisionTreeClassifier(max_depth=3,random_state=0)
dt.fit(X_train,y_train)
y_pred = dt.predict(X_test)
print("dt_accuracy_score : {}".format(accuracy_score(y_test,y_pred)))
 
 
#ランダムフォレスト
from sklearn.ensemble import RandomForestClassifier
 
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
print("rf_accuracy_score : {}".format(accuracy_score(y_test,y_pred)))
 
#XGBoost
import xgboost as xgb
from xgboost import XGBClassifier
 
xgb = xgb.XGBClassifier(booster="gbtree",silent=0,seed=0)
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
print("xgb_accuracy_score : {}".format(accuracy_score(y_test,y_pred)))

#3つのモデルの特徴量重要度を並べてみる
models = ["DT","RF","XGB"]
importances = pd.DataFrame({"features":X_train.columns.values,
                            models[0]:dt.feature_importances_,
                            models[1]:rf.feature_importances_,
                            models[2]:xgb.feature_importances_})
 
#データフレームをDTの降順に並び替えて表示
importances.sort_values("DT",ascending=False)

dt_accuracy_score : 0.7575757575757576
rf_accuracy_score : 0.7575757575757576
xgb_accuracy_score : 0.7575757575757576


,features,DT,RF,XGB
39,配管ID_P450S40,0.506138,0.048972,0.000000
0,呼び径 (mm)_x,0.329308,0.123337,0.601403
38,配管ID_P400S40,0.164554,0.053236,0.000000
36,配管ID_P300S40,0.000000,0.063923,0.000000
27,周方向弾性係数 (E6 N/mm2),0.000000,0.000000,0.000000
28,せん断弾性係数 (E6 N/mm2),0.000000,0.000000,0.000000
29,密度 (kg/m3),0.000000,0.000000,0.000000
30,ﾎﾟｱｿﾝ比,0.000000,0.000000,0.000000
31,材質_STPG370-S,0.000000,0.000000,0.000000
32,配管 ID_P300S40,0.000000,0.075438,0.000000


In [0]:
#P36

#圧力_温度_配管ID.csvの読み込み
data1 = pd.read_csv("drive/My Drive/mdb/test/P36/圧力_温度_配管ID.csv", encoding='shift_jis')
#配管特性.csvの読み込み
data2 = pd.read_csv('drive/My Drive/mdb/test/P36/配管特性.csv',encoding='shift_jis')
#圧力_温度_配管ID.csvのデータの配管IDに合わせて配管特性.csvのデータを結合
data1_frame = DataFrame(data1)
data2_frame = DataFrame(data2)
data12 = pd.merge(data1_frame,data2_frame,how='left',left_on='配管 ID',right_on = '配管ID')
#'To'データの削除
data12 = data12.drop("To",axis=1)
for col_name in data12.columns:
    print(col_name,data12[col_name].dtype)
#'配管ID'などの文字データのダミー化
data12_dummy = pd.get_dummies(data12[data12.columns[data12.columns != 'From']])
data12_dummy['From'] = data12['From']
#Support.csvの読み込み
data_out = pd.read_csv("drive/My Drive/mdb/test/P36/Support.csv",encoding='shift_jis')
#使用したい "Point","Type"データのみにする
for col_name in data_out.columns:
    if col_name != "Point" and col_name != "Type" :
        data_out = data_out.drop(col_name,axis=1)
#重複しているデータの削除
data_out = data_out.drop_duplicates()
#入力データと出力データを'Point'データで結合
data_press_type = pd.merge(data12_dummy,data_out,how = "outer",left_on = 'From',right_on='Point')
#文字データなのでダミー化
data_press_type.at[data_press_type['Type'] == "Guide",'Type'] = 1
data_press_type.at[data_press_type['Type'] == "V - Stop",'Type'] = 2
data_press_type.at[data_press_type['Type'] == "Spr. Hgr",'Type'] = 3
data_press_type.at[data_press_type['Type'] == "Inclined",'Type'] = 4
data_test_press = data_press_type.fillna(0)
data_series = Series(data_test_press['Type'])
print(pd.Series.value_counts(data_series))
data_test_press = data_test_press[data_test_press["呼び径 (mm)_x"] != 0]
data_test_press

From object
材質 object
配管 ID object
呼び径 (mm)_x float64
ｹｰｽ 1 圧力 (N/mm2) float64
ｹｰｽ 1 温度 (deg C) float64
ｹｰｽ 1 自動_1 float64
ｹｰｽ 1 線膨張量 (mm/m) float64
ｹｰｽ 1 自動_2 float64
ｹｰｽ 1 運転時弾性係数 (E6 N/mm2) float64
ｹｰｽ 1 自動_3 float64
ｹｰｽ 1 運転時許容応力 (N/mm2) float64
配管ID object
ﾀｸﾞ ﾅﾝﾊﾞｰ object
呼び径 (mm)_y int64
実外径 (mm) float64
ｽｹｼﾞｭｰﾙ int64
肉厚 (mm) float64
腐れ代 (mm) float64
製作公差 (mm) float64
保温厚さ (mm) float64
保温材質 object
保温材密度 (kg/m3) float64
外装板厚 (mm) float64
外装板材質 object
外装板密度 (kg/m3) object
ﾗｲﾆﾝｸﾞ厚さ (mm) float64
ﾗｲﾆﾝｸﾞ密度 (kg/m3) object
ﾗｲﾝ ｸﾗｽ object
流体比重 float64
配管材質 object
最低温度の警告 float64
溶接E係数 float64
溶接W係数 自動 object
溶接W係数 object
応力振幅 低減係数 float64
常温時許容応力 (N/mm2) float64
最小降伏点(Sy) (N/mm2) float64
引張強さ(Su) (N/mm2) float64
縦弾性係数 (E6 N/mm2) float64
周方向弾性係数 (E6 N/mm2) float64
せん断弾性係数 (E6 N/mm2) float64
密度 (kg/m3) float64
ﾎﾟｱｿﾝ比 float64
0    98
1    17
2     6
4     2
3     1
Name: Type, dtype: int64


,呼び径 (mm)_x,ｹｰｽ 1 圧力 (N/mm2),ｹｰｽ 1 温度 (deg C),ｹｰｽ 1 自動_1,ｹｰｽ 1 線膨張量 (mm/m),ｹｰｽ 1 自動_2,ｹｰｽ 1 運転時弾性係数 (E6 N/mm2),ｹｰｽ 1 自動_3,ｹｰｽ 1 運転時許容応力 (N/mm2),呼び径 (mm)_y,実外径 (mm),ｽｹｼﾞｭｰﾙ,肉厚 (mm),腐れ代 (mm),製作公差 (mm),保温厚さ (mm),保温材密度 (kg/m3),外装板厚 (mm),ﾗｲﾆﾝｸﾞ厚さ (mm),流体比重,最低温度の警告,溶接E係数,応力振幅 低減係数,常温時許容応力 (N/mm2),最小降伏点(Sy) (N/mm2),引張強さ(Su) (N/mm2),縦弾性係数 (E6 N/mm2),周方向弾性係数 (E6 N/mm2),せん断弾性係数 (E6 N/mm2),密度 (kg/m3),ﾎﾟｱｿﾝ比,材質_STPG370-S,配管 ID_P100S40,配管 ID_P125S40,配管 ID_P150S40,配管 ID_P200S40,配管ID_P100S40,配管ID_P125S40,配管ID_P150S40,配管ID_P200S40,ﾀｸﾞ ﾅﾝﾊﾞｰ_,保温材質_Other,外装板材質_,外装板密度 (kg/m3)_,ﾗｲﾆﾝｸﾞ密度 (kg/m3)_,ﾗｲﾝ ｸﾗｽ_BB22,配管材質_STPG370-S,溶接W係数 自動_,溶接W係数_,From,Point,Type
0,200.0,2.40,112.0,1.0,1.05,1.0,0.19,1.0,106.0,200.0,216.3,40.0,8.20,0.0,1.02,30.0,201.77,0.0,0.0,0.99,0.0,1.0,1.0,92.0,215.0,370.0,0.2,0.2,0.07,7850.0,0.3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,A00,0,0
1,200.0,2.40,112.0,1.0,1.05,1.0,0.19,1.0,106.0,200.0,216.3,40.0,8.20,0.0,1.02,30.0,201.77,0.0,0.0,0.99,0.0,1.0,1.0,92.0,215.0,370.0,0.2,0.2,0.07,7850.0,0.3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,A01,0,0
2,200.0,2.40,112.0,1.0,1.05,1.0,0.19,1.0,106.0,200.0,216.3,40.0,8.20,0.0,1.02,30.0,201.77,0.0,0.0,0.99,0.0,1.0,1.0,92.0,215.0,370.0,0.2,0.2,0.07,7850.0,0.3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,A02,0,0
3,150.0,2.40,112.0,1.0,1.05,1.0,0.19,1.0,106.0,150.0,165.2,40.0,7.09,0.0,0.88,30.0,201.39,0.0,0.0,0.99,0.0,1.0,1.0,92.0,215.0,370.0,0.2,0.2,0.07,7850.0,0.3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,A03,0,0
4,150.0,2.40,112.0,1.0,1.05,1.0,0.19,1.0,106.0,150.0,165.2,40.0,7.09,0.0,0.88,30.0,201.39,0.0,0.0,0.99,0.0,1.0,1.0,92.0,215.0,370.0,0.2,0.2,0.07,7850.0,0.3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,A04 N,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,125.0,0.08,120.0,1.0,1.15,1.0,0.19,1.0,106.0,125.0,139.8,40.0,6.60,0.0,0.82,30.0,201.22,0.0,0.0,1.00,0.0,1.0,1.0,92.0,215.0,370.0,0.2,0.2,0.07,7850.0,0.3,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,B45 N,0,0
116,125.0,0.08,120.0,1.0,1.15,1.0,0.19,1.0,106.0,125.0,139.8,40.0,6.60,0.0,0.82,30.0,201.22,0.0,0.0,1.00,0.0,1.0,1.0,92.0,215.0,370.0,0.2,0.2,0.07,7850.0,0.3,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,B45 F,0,0
117,125.0,0.08,120.0,1.0,1.15,1.0,0.19,1.0,106.0,125.0,139.8,40.0,6.60,0.0,0.82,30.0,201.22,0.0,0.0,1.00,0.0,1.0,1.0,92.0,215.0,370.0,0.2,0.2,0.07,7850.0,0.3,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,B46 N,0,0
118,125.0,0.08,120.0,1.0,1.15,1.0,0.19,1.0,106.0,125.0,139.8,40.0,6.60,0.0,0.82,30.0,201.22,0.0,0.0,1.00,0.0,1.0,1.0,92.0,215.0,370.0,0.2,0.2,0.07,7850.0,0.3,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,B46 F,0,0


In [0]:
np.random.seed(0) # 乱数を固定値で初期化し再現性を持たせる
X = data_test_press.drop(['From','Point','Type'],axis=1)
T = data_test_press['Type']

X_train,X_test, y_train, y_test = train_test_split(X, T, train_size=0.7, test_size=0.3)

from sklearn.metrics import confusion_matrix,accuracy_score
 
#決定木
from sklearn.tree import DecisionTreeClassifier
 
dt = DecisionTreeClassifier(max_depth=3,random_state=0)
dt.fit(X_train,y_train)
y_pred = dt.predict(X_test)
print("dt_accuracy_score : {}".format(accuracy_score(y_test,y_pred)))
 
 
#ランダムフォレスト
from sklearn.ensemble import RandomForestClassifier
 
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
print("rf_accuracy_score : {}".format(accuracy_score(y_test,y_pred)))
 
#XGBoost
import xgboost as xgb
from xgboost import XGBClassifier
 
xgb = xgb.XGBClassifier(booster="gbtree",silent=0,seed=0)
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
print("xgb_accuracy_score : {}".format(accuracy_score(y_test,y_pred)))

#3つのモデルの特徴量重要度を並べてみる
models = ["DT","RF","XGB"]
importances = pd.DataFrame({"features":X_train.columns.values,
                            models[0]:dt.feature_importances_,
                            models[1]:rf.feature_importances_,
                            models[2]:xgb.feature_importances_})
 
#データフレームをDTの降順に並び替えて表示
importances.sort_values("DT",ascending=False)

dt_accuracy_score : 0.8611111111111112
rf_accuracy_score : 0.8611111111111112
xgb_accuracy_score : 0.8611111111111112


,features,DT,RF,XGB
2,ｹｰｽ 1 温度 (deg C),0.711674,0.131257,0.000000
1,ｹｰｽ 1 圧力 (N/mm2),0.288326,0.328313,0.853393
37,配管ID_P125S40,0.000000,0.069944,0.000000
27,周方向弾性係数 (E6 N/mm2),0.000000,0.000000,0.000000
28,せん断弾性係数 (E6 N/mm2),0.000000,0.000000,0.000000
29,密度 (kg/m3),0.000000,0.000000,0.000000
30,ﾎﾟｱｿﾝ比,0.000000,0.000000,0.000000
31,材質_STPG370-S,0.000000,0.000000,0.000000
32,配管 ID_P100S40,0.000000,0.001150,0.000000
33,配管 ID_P125S40,0.000000,0.061456,0.000000


In [0]:
#Q10

#圧力_温度_配管ID.csvの読み込み
data1 = pd.read_csv("drive/My Drive/mdb/test/Q10/圧力_温度_配管ID.csv", encoding='shift_jis')
#配管特性.csvの読み込み
data2 = pd.read_csv('drive/My Drive/mdb/test/Q10/配管特性.csv',encoding='shift_jis')
#圧力_温度_配管ID.csvのデータの配管IDに合わせて配管特性.csvのデータを結合
data1_frame = DataFrame(data1)
data2_frame = DataFrame(data2)
data12 = pd.merge(data1_frame,data2_frame,how='left',left_on='配管 ID',right_on = '配管ID')
#'To'データの削除
data12 = data12.drop("To",axis=1)
for col_name in data12.columns:
    print(col_name,data12[col_name].dtype)
#'配管ID'などの文字データのダミー化
data12_dummy = pd.get_dummies(data12[data12.columns[data12.columns != 'From']])
data12_dummy['From'] = data12['From']
#Support.csvの読み込み
data_out = pd.read_csv("drive/My Drive/mdb/test/Q10/Support.csv",encoding='shift_jis')
#使用したい "Point","Type"データのみにする
for col_name in data_out.columns:
    if col_name != "Point" and col_name != "Type" :
        data_out = data_out.drop(col_name,axis=1)
#重複しているデータの削除
data_out = data_out.drop_duplicates()
#入力データと出力データを'Point'データで結合
data_press_type = pd.merge(data12_dummy,data_out,how = "outer",left_on = 'From',right_on='Point')
#文字データなのでダミー化
data_press_type.at[data_press_type['Type'] == "Guide",'Type'] = 1
data_press_type.at[data_press_type['Type'] == "V - Stop",'Type'] = 2
data_test_press = data_press_type.fillna(0)
data_series = Series(data_test_press['Type'])
print(pd.Series.value_counts(data_series))
data_test_press = data_test_press[data_test_press["呼び径 (mm)_x"] != 0]
data_test_press

From object
材質 object
配管 ID object
呼び径 (mm)_x int64
ｹｰｽ 1 圧力 (N/mm2) float64
ｹｰｽ 1 温度 (deg C) int64
ｹｰｽ 1 自動_1 int64
ｹｰｽ 1 線膨張量 (mm/m) float64
ｹｰｽ 1 自動_2 int64
ｹｰｽ 1 運転時弾性係数 (E6 N/mm2) float64
ｹｰｽ 1 自動_3 int64
ｹｰｽ 1 運転時許容応力 (N/mm2) int64
配管ID object
ﾀｸﾞ ﾅﾝﾊﾞｰ object
呼び径 (mm)_y int64
実外径 (mm) float64
ｽｹｼﾞｭｰﾙ object
肉厚 (mm) float64
腐れ代 (mm) int64
製作公差 (mm) float64
保温厚さ (mm) int64
保温材質 object
保温材密度 (kg/m3) object
外装板厚 (mm) int64
外装板材質 object
外装板密度 (kg/m3) object
ﾗｲﾆﾝｸﾞ厚さ (mm) int64
ﾗｲﾆﾝｸﾞ密度 (kg/m3) object
ﾗｲﾝ ｸﾗｽ object
流体比重 float64
配管材質 object
最低温度の警告 int64
溶接E係数 int64
溶接W係数 自動 object
溶接W係数 object
応力振幅 低減係数 int64
常温時許容応力 (N/mm2) float64
最小降伏点(Sy) (N/mm2) int64
引張強さ(Su) (N/mm2) int64
縦弾性係数 (E6 N/mm2) float64
周方向弾性係数 (E6 N/mm2) float64
せん断弾性係数 (E6 N/mm2) float64
密度 (kg/m3) int64
ﾎﾟｱｿﾝ比 float64
0    219
1     33
2     13
Name: Type, dtype: int64


,呼び径 (mm)_x,ｹｰｽ 1 圧力 (N/mm2),ｹｰｽ 1 温度 (deg C),ｹｰｽ 1 自動_1,ｹｰｽ 1 線膨張量 (mm/m),ｹｰｽ 1 自動_2,ｹｰｽ 1 運転時弾性係数 (E6 N/mm2),ｹｰｽ 1 自動_3,ｹｰｽ 1 運転時許容応力 (N/mm2),呼び径 (mm)_y,実外径 (mm),肉厚 (mm),腐れ代 (mm),製作公差 (mm),保温厚さ (mm),外装板厚 (mm),ﾗｲﾆﾝｸﾞ厚さ (mm),流体比重,最低温度の警告,溶接E係数,応力振幅 低減係数,常温時許容応力 (N/mm2),最小降伏点(Sy) (N/mm2),引張強さ(Su) (N/mm2),縦弾性係数 (E6 N/mm2),周方向弾性係数 (E6 N/mm2),せん断弾性係数 (E6 N/mm2),密度 (kg/m3),ﾎﾟｱｿﾝ比,材質_SGP-B,材質_SGP-E,材質_STPG370-S,材質_STPT370-S,配管 ID_P100S40,配管 ID_P100SGPB,配管 ID_P150SGPE,配管 ID_P200SGPE,配管 ID_P50S40,配管ID_P100S40,配管ID_P100SGPB,配管ID_P150SGPE,配管ID_P200SGPE,配管ID_P50S40,ﾀｸﾞ ﾅﾝﾊﾞｰ_,ｽｹｼﾞｭｰﾙ_40,ｽｹｼﾞｭｰﾙ_SGP,保温材質_,保温材密度 (kg/m3)_,外装板材質_,外装板密度 (kg/m3)_,ﾗｲﾆﾝｸﾞ密度 (kg/m3)_,ﾗｲﾝ ｸﾗｽ_,配管材質_SGP-B,配管材質_SGP-E,配管材質_STPG370-S,配管材質_STPT370-S,溶接W係数 自動_,溶接W係数_,From,Point,Type
0,150,0.1,100,1,0.91,1,0.19,1,62,150,165.2,5.0,0,0.62,0,0,0,1.00,0,1,1,62.00,147,290,0.2,0.2,0.07,7850,0.3,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,1,1,1,1,1,1,0,1,0,0,1,1,A00,0,0
1,150,0.1,100,1,0.91,1,0.19,1,62,150,165.2,5.0,0,0.62,0,0,0,1.00,0,1,1,62.00,147,290,0.2,0.2,0.07,7850,0.3,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,1,1,1,1,1,1,0,1,0,0,1,1,A01 N,0,0
2,150,0.1,100,1,0.91,1,0.19,1,62,150,165.2,5.0,0,0.62,0,0,0,1.00,0,1,1,62.00,147,290,0.2,0.2,0.07,7850,0.3,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,1,1,1,1,1,1,0,1,0,0,1,1,A01 F,0,0
3,150,0.1,100,1,0.91,1,0.19,1,62,150,165.2,5.0,0,0.62,0,0,0,1.00,0,1,1,62.00,147,290,0.2,0.2,0.07,7850,0.3,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,1,1,1,1,1,1,0,1,0,0,1,1,A02,A02,1
4,150,0.1,100,1,0.91,1,0.19,1,62,150,165.2,5.0,0,0.62,0,0,0,1.00,0,1,1,62.00,147,290,0.2,0.2,0.07,7850,0.3,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,1,1,1,1,1,1,0,1,0,0,1,1,A03 N,0,0
5,150,0.1,100,1,0.91,1,0.19,1,62,150,165.2,5.0,0,0.62,0,0,0,1.00,0,1,1,62.00,147,290,0.2,0.2,0.07,7850,0.3,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,1,1,1,1,1,1,0,1,0,0,1,1,A03 F,0,0
6,150,0.1,100,1,0.91,1,0.19,1,62,150,165.2,5.0,0,0.62,0,0,0,1.00,0,1,1,62.00,147,290,0.2,0.2,0.07,7850,0.3,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,1,1,1,1,1,1,0,1,0,0,1,1,A04,A04,2
7,150,0.1,100,1,0.91,1,0.19,1,62,150,165.2,5.0,0,0.62,0,0,0,1.00,0,1,1,62.00,147,290,0.2,0.2,0.07,7850,0.3,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,1,1,1,1,1,1,0,1,0,0,1,1,A05 N,0,0
8,150,0.1,100,1,0.91,1,0.19,1,62,150,165.2,5.0,0,0.62,0,0,0,1.00,0,1,1,62.00,147,290,0.2,0.2,0.07,7850,0.3,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,1,1,1,1,1,1,0,1,0,0,1,1,A05 F,0,0
9,150,0.1,100,1,0.91,1,0.19,1,62,150,165.2,5.0,0,0.62,0,0,0,1.00,0,1,1,62.00,147,290,0.2,0.2,0.07,7850,0.3,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,1,1,1,1,1,1,0,1,0,0,1,1,A06,A06,2


In [0]:
np.random.seed(0) # 乱数を固定値で初期化し再現性を持たせる
X = data_test_press.drop(['From','Point','Type'],axis=1)
T = data_test_press['Type']

X_train,X_test, y_train, y_test = train_test_split(X, T, train_size=0.7, test_size=0.3)

from sklearn.metrics import confusion_matrix,accuracy_score
 
#決定木
from sklearn.tree import DecisionTreeClassifier
 
dt = DecisionTreeClassifier(max_depth=3,random_state=0)
dt.fit(X_train,y_train)
y_pred = dt.predict(X_test)
print("dt_accuracy_score : {}".format(accuracy_score(y_test,y_pred)))
 
 
#ランダムフォレスト
from sklearn.ensemble import RandomForestClassifier
 
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
print("rf_accuracy_score : {}".format(accuracy_score(y_test,y_pred)))
 
#XGBoost
import xgboost as xgb
from xgboost import XGBClassifier
 
xgb = xgb.XGBClassifier(booster="gbtree",silent=0,seed=0)
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
print("xgb_accuracy_score : {}".format(accuracy_score(y_test,y_pred)))

#3つのモデルの特徴量重要度を並べてみる
models = ["DT","RF","XGB"]
importances = pd.DataFrame({"features":X_train.columns.values,
                            models[0]:dt.feature_importances_,
                            models[1]:rf.feature_importances_,
                            models[2]:xgb.feature_importances_})
 
#データフレームをDTの降順に並び替えて表示
importances.sort_values("DT",ascending=False)

dt_accuracy_score : 0.8125
rf_accuracy_score : 0.8125
xgb_accuracy_score : 0.8125


,features,DT,RF,XGB
8,ｹｰｽ 1 運転時許容応力 (N/mm2),0.578701,0.077112,0.514241
2,ｹｰｽ 1 温度 (deg C),0.268578,0.084507,0.303488
53,配管材質_SGP-E,0.133447,0.012893,0.000000
33,配管 ID_P100S40,0.011855,0.005046,0.000000
0,呼び径 (mm)_x,0.007417,0.020936,0.026853
43,ﾀｸﾞ ﾅﾝﾊﾞｰ_,0.000000,0.000000,0.000000
32,材質_STPT370-S,0.000000,0.018134,0.000000
34,配管 ID_P100SGPB,0.000000,0.026820,0.000000
35,配管 ID_P150SGPE,0.000000,0.008948,0.084900
36,配管 ID_P200SGPE,0.000000,0.020620,0.000000


In [4]:
#C50&Q10

#C50

#圧力_温度_配管ID.csvの読み込み
data1 = pd.read_csv("drive/My Drive/mdb/test/C50/圧力_温度_配管ID.csv", encoding='shift_jis')
#配管特性.csvの読み込み
data2 = pd.read_csv('drive/My Drive/mdb/test/C50/配管特性.csv',encoding='shift_jis')
#圧力_温度_配管ID.csvのデータの配管IDに合わせて配管特性.csvのデータを結合
data1_frame = DataFrame(data1)
data2_frame = DataFrame(data2)
data12 = pd.merge(data1_frame,data2_frame,how='left',left_on='配管 ID',right_on = '配管ID')
#'To'データの削除
data12 = data12.drop("To",axis=1)
for col_name in data12.columns:
    print(col_name,data12[col_name].dtype)
#'配管ID'などの文字データのダミー化
data12_dummy = pd.get_dummies(data12[data12.columns[data12.columns != 'From']])
data12_dummy['From'] = data12['From']
print(data12_dummy['配管 ID_P250S40'])
#Support.csvの読み込み
data_out = pd.read_csv("drive/My Drive/mdb/test/C50/Support.csv",encoding='shift_jis')
#使用したい "Point","Type"データのみにする
for col_name in data_out.columns:
    if col_name != "Point" and col_name != "Type" :
        data_out = data_out.drop(col_name,axis=1)
#重複しているデータの削除
data_out = data_out.drop_duplicates()
#入力データと出力データを'Point'データで結合
data_press_type = pd.merge(data12_dummy,data_out,how = "outer",left_on = 'From',right_on='Point')
#'Support.csv'の'type'データの確認
data_series = Series(data_press_type['Type'])
pd.Series.value_counts(data_series)
#文字データなのでダミー化
data_press_type.at[data_press_type['Type'] == "Guide",'Type'] = 1
data_press_type.at[data_press_type['Type'] == "V - Stop",'Type'] = 2
data_press_type.at[data_press_type['Type'] == "Spr. Hgr",'Type'] = 3
data_press_type.at[data_press_type['Type'] == "Spr. Can",'Type'] = 4
data_test_press = data_press_type.fillna(0)
data_series = Series(data_test_press['Type'])
print(pd.Series.value_counts(data_series))
data_test_press_c50 = data_test_press[data_test_press["呼び径 (mm)_x"] != 0]
data_test_press_c50 = pd.DataFrame(data_test_press_c50)

#Q10

#圧力_温度_配管ID.csvの読み込み
data1 = pd.read_csv("drive/My Drive/mdb/test/Q10/圧力_温度_配管ID.csv", encoding='shift_jis')
#配管特性.csvの読み込み
data2 = pd.read_csv('drive/My Drive/mdb/test/Q10/配管特性.csv',encoding='shift_jis')
#圧力_温度_配管ID.csvのデータの配管IDに合わせて配管特性.csvのデータを結合
data1_frame = DataFrame(data1)
data2_frame = DataFrame(data2)
data12 = pd.merge(data1_frame,data2_frame,how='left',left_on='配管 ID',right_on = '配管ID')
#'To'データの削除
data12 = data12.drop("To",axis=1)
for col_name in data12.columns:
    print(col_name,data12[col_name].dtype)
#'配管ID'などの文字データのダミー化
data12_dummy = pd.get_dummies(data12[data12.columns[data12.columns != 'From']])
data12_dummy['From'] = data12['From']
#Support.csvの読み込み
data_out = pd.read_csv("drive/My Drive/mdb/test/Q10/Support.csv",encoding='shift_jis')
#使用したい "Point","Type"データのみにする
for col_name in data_out.columns:
    if col_name != "Point" and col_name != "Type" :
        data_out = data_out.drop(col_name,axis=1)
#重複しているデータの削除
data_out = data_out.drop_duplicates()
#入力データと出力データを'Point'データで結合
data_press_type = pd.merge(data12_dummy,data_out,how = "outer",left_on = 'From',right_on='Point')
#'Support.csv'の'type'データの確認
data_series = Series(data_press_type['Type'])
pd.Series.value_counts(data_series)
#文字データなのでダミー化
data_press_type.at[data_press_type['Type'] == "Guide",'Type'] = 1
data_press_type.at[data_press_type['Type'] == "V - Stop",'Type'] = 2
data_test_press = data_press_type.fillna(0)
data_series = Series(data_test_press['Type'])
print(pd.Series.value_counts(data_series))
data_test_press_q10 = data_test_press[data_test_press["呼び径 (mm)_x"] != 0]
data_test_press_q10 = pd.DataFrame(data_test_press_q10)

#データの結合
data_test_press = pd.concat([data_test_press_c50,data_test_press_q10])
data_test_press = data_test_press.fillna(0)
data_test_press

From object
材質 object
配管 ID object
呼び径 (mm)_x int64
ｹｰｽ 1 圧力 (N/mm2) float64
ｹｰｽ 1 温度 (deg C) int64
ｹｰｽ 1 自動_1 int64
ｹｰｽ 1 線膨張量 (mm/m) float64
ｹｰｽ 1 自動_2 int64
ｹｰｽ 1 運転時弾性係数 (E6 N/mm2) float64
ｹｰｽ 1 自動_3 int64
ｹｰｽ 1 運転時許容応力 (N/mm2) float64
配管ID object
ﾀｸﾞ ﾅﾝﾊﾞｰ object
呼び径 (mm)_y int64
実外径 (mm) float64
ｽｹｼﾞｭｰﾙ object
肉厚 (mm) float64
腐れ代 (mm) int64
製作公差 (mm) float64
保温厚さ (mm) int64
保温材質 object
保温材密度 (kg/m3) float64
外装板厚 (mm) int64
外装板材質 object
外装板密度 (kg/m3) object
ﾗｲﾆﾝｸﾞ厚さ (mm) int64
ﾗｲﾆﾝｸﾞ密度 (kg/m3) object
ﾗｲﾝ ｸﾗｽ object
流体比重 int64
配管材質 object
最低温度の警告 int64
溶接E係数 int64
溶接W係数 自動 object
溶接W係数 object
応力振幅 低減係数 int64
常温時許容応力 (N/mm2) float64
最小降伏点(Sy) (N/mm2) int64
引張強さ(Su) (N/mm2) int64
縦弾性係数 (E6 N/mm2) float64
周方向弾性係数 (E6 N/mm2) float64
せん断弾性係数 (E6 N/mm2) float64
密度 (kg/m3) int64
ﾎﾟｱｿﾝ比 float64
0      1
1      1
2      1
3      1
4      1
      ..
333    0
334    0
335    0
336    0
337    0
Name: 配管 ID_P250S40, Length: 338, dtype: uint8
0    277
1     35
2     17
3     12
4      1
Name: T

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:78: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,From,Point,Type,せん断弾性係数 (E6 N/mm2),保温厚さ (mm),保温材密度 (kg/m3),保温材密度 (kg/m3)_,保温材質_,保温材質_Other,周方向弾性係数 (E6 N/mm2),呼び径 (mm)_x,呼び径 (mm)_y,外装板厚 (mm),外装板密度 (kg/m3)_,外装板材質_,実外径 (mm),密度 (kg/m3),常温時許容応力 (N/mm2),引張強さ(Su) (N/mm2),応力振幅 低減係数,最低温度の警告,最小降伏点(Sy) (N/mm2),材質_SB410,材質_SGP-B,材質_SGP-E,材質_STPG370-S,材質_STPT370-S,流体比重,溶接E係数,溶接W係数 自動_,溶接W係数_,縦弾性係数 (E6 N/mm2),肉厚 (mm),腐れ代 (mm),製作公差 (mm),配管 ID_E350S40,配管 ID_E400T13,配管 ID_E500T13,配管 ID_P100S40,配管 ID_P100SGPB,配管 ID_P125S40,配管 ID_P150S40,配管 ID_P150SGPE,配管 ID_P200S40,配管 ID_P200SGPE,配管 ID_P250S40,配管 ID_P350S40,配管 ID_P450S40,配管 ID_P450STD,配管 ID_P450T13,配管 ID_P50S40,配管 ID_P550STD,配管 ID_P600STD,配管 ID_P600T13,配管 ID_P700STD,配管ID_E350S40,配管ID_E400T13,配管ID_E500T13,配管ID_P100S40,配管ID_P100SGPB,配管ID_P125S40,配管ID_P150S40,配管ID_P150SGPE,配管ID_P200S40,配管ID_P200SGPE,配管ID_P250S40,配管ID_P350S40,配管ID_P450S40,配管ID_P450STD,配管ID_P450T13,配管ID_P50S40,配管ID_P550STD,配管ID_P600STD,配管ID_P600T13,配管ID_P700STD,配管材質_SB410,配管材質_SGP-B,配管材質_SGP-E,配管材質_STPG370-S,配管材質_STPT370-S,ｹｰｽ 1 圧力 (N/mm2),ｹｰｽ 1 温度 (deg C),ｹｰｽ 1 線膨張量 (mm/m),ｹｰｽ 1 自動_1,ｹｰｽ 1 自動_2,ｹｰｽ 1 自動_3,ｹｰｽ 1 運転時弾性係数 (E6 N/mm2),ｹｰｽ 1 運転時許容応力 (N/mm2),ｽｹｼﾞｭｰﾙ_40,ｽｹｼﾞｭｰﾙ_NS,ｽｹｼﾞｭｰﾙ_SGP,ｽｹｼﾞｭｰﾙ_STD,ﾀｸﾞ ﾅﾝﾊﾞｰ_,ﾎﾟｱｿﾝ比,ﾗｲﾆﾝｸﾞ厚さ (mm),ﾗｲﾆﾝｸﾞ密度 (kg/m3)_,ﾗｲﾝ ｸﾗｽ_,ﾗｲﾝ ｸﾗｽ_BC22,ﾗｲﾝ ｸﾗｽ_BC23,ﾗｲﾝ ｸﾗｽ_BC24
0,A00,0,0,0.078049,65.0,201.47,0.0,0.0,1.0,0.202927,250.0,250.0,0.0,1.0,1.0,267.4,7850.0,92.00,370.0,1.0,0.0,215.0,0.0,0.0,0.0,0.0,1.0,0.00,1.0,1.0,1.0,0.202927,9.3,0.0,1.1625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.77,375.0,4.750695,1.0,1.0,1.0,0.1745,89.0,1.0,0.0,0.0,0.0,1.0,0.3,0.0,1.0,0.0,1.0,0.0,0.0
1,A01,0,0,0.078049,65.0,201.47,0.0,0.0,1.0,0.202927,250.0,250.0,0.0,1.0,1.0,267.4,7850.0,92.00,370.0,1.0,0.0,215.0,0.0,0.0,0.0,0.0,1.0,0.00,1.0,1.0,1.0,0.202927,9.3,0.0,1.1625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.77,375.0,4.750695,1.0,1.0,1.0,0.1745,89.0,1.0,0.0,0.0,0.0,1.0,0.3,0.0,1.0,0.0,1.0,0.0,0.0
2,A02 N,0,0,0.078049,65.0,201.47,0.0,0.0,1.0,0.202927,250.0,250.0,0.0,1.0,1.0,267.4,7850.0,92.00,370.0,1.0,0.0,215.0,0.0,0.0,0.0,0.0,1.0,0.00,1.0,1.0,1.0,0.202927,9.3,0.0,1.1625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.77,375.0,4.750695,1.0,1.0,1.0,0.1745,89.0,1.0,0.0,0.0,0.0,1.0,0.3,0.0,1.0,0.0,1.0,0.0,0.0
3,A02 F,0,0,0.078049,65.0,201.47,0.0,0.0,1.0,0.202927,250.0,250.0,0.0,1.0,1.0,267.4,7850.0,92.00,370.0,1.0,0.0,215.0,0.0,0.0,0.0,0.0,1.0,0.00,1.0,1.0,1.0,0.202927,9.3,0.0,1.1625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.77,375.0,4.750695,1.0,1.0,1.0,0.1745,89.0,1.0,0.0,0.0,0.0,1.0,0.3,0.0,1.0,0.0,1.0,0.0,0.0
4,A03 N,0,0,0.078049,65.0,201.47,0.0,0.0,1.0,0.202927,250.0,250.0,0.0,1.0,1.0,267.4,7850.0,92.00,370.0,1.0,0.0,215.0,0.0,0.0,0.0,0.0,1.0,0.00,1.0,1.0,1.0,0.202927,9.3,0.0,1.1625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.77,375.0,4.750695,1.0,1.0,1.0,0.1745,89.0,1.0,0.0,0.0,0.0,1.0,0.3,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,Q05 N,0,0,0.070000,0.0,0.00,1.0,1.0,0.0,0.200000,50.0,50.0,0.0,1.0,1.0,60.5,7850.0,9

In [5]:
np.random.seed(0) # 乱数を固定値で初期化し再現性を持たせる
X = data_test_press.drop(['From','Point','Type'],axis=1)
T = data_test_press['Type']

X_train,X_test, y_train, y_test = train_test_split(X, T, train_size=0.7, test_size=0.3)

from sklearn.metrics import confusion_matrix,accuracy_score
 
#決定木
from sklearn.tree import DecisionTreeClassifier
 
dt = DecisionTreeClassifier(max_depth=3,random_state=0)
dt.fit(X_train,y_train)
y_pred = dt.predict(X_test)
print("dt_accuracy_score : {}".format(accuracy_score(y_test,y_pred)))
 
 
#ランダムフォレスト
from sklearn.ensemble import RandomForestClassifier
 
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
print("rf_accuracy_score : {}".format(accuracy_score(y_test,y_pred)))
 
#XGBoost
import xgboost as xgb
from xgboost import XGBClassifier
 
xgb = xgb.XGBClassifier(booster="gbtree",silent=0,seed=0)
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
print("xgb_accuracy_score : {}".format(accuracy_score(y_test,y_pred)))

#3つのモデルの特徴量重要度を並べてみる
models = ["DT","RF","XGB"]
importances = pd.DataFrame({"features":X_train.columns.values,
                            models[0]:dt.feature_importances_,
                            models[1]:rf.feature_importances_,
                            models[2]:xgb.feature_importances_})
 
#データフレームをDTの降順に並び替えて表示
importances.sort_values("DT",ascending=False)

dt_accuracy_score : 0.8287292817679558
rf_accuracy_score : 0.8287292817679558
xgb_accuracy_score : 0.8287292817679558


,features,DT,RF,XGB
84,ｹｰｽ 1 運転時許容応力 (N/mm2),0.438112,0.034934,0.071598
66,配管ID_P450T13,0.227366,0.017596,0.000000
37,配管 ID_P125S40,0.146365,0.021493,0.065923
83,ｹｰｽ 1 運転時弾性係数 (E6 N/mm2),0.117261,0.037845,0.000000
65,配管ID_P450STD,0.070896,0.006891,0.000000
69,配管ID_P600STD,0.000000,0.002198,0.000000
68,配管ID_P550STD,0.000000,0.004689,0.000000
67,配管ID_P50S40,0.000000,0.003654,0.000000
64,配管ID_P450S40,0.000000,0.001299,0.000000
63,配管ID_P350S40,0.000000,0.008271,0.000000


In [8]:
#C61&L41&P36

#C61

#圧力_温度_配管ID.csvの読み込み
data1 = pd.read_csv("drive/My Drive/mdb/test/C61/圧力_温度_配管ID.csv", encoding='shift_jis')
#配管特性.csvの読み込み
data2 = pd.read_csv('drive/My Drive/mdb/test/C61/配管特性.csv',encoding='shift_jis')
#圧力_温度_配管ID.csvのデータの配管IDに合わせて配管特性.csvのデータを結合
data1_frame = DataFrame(data1)
data2_frame = DataFrame(data2)
data12 = pd.merge(data1_frame,data2_frame,how='left',left_on='配管 ID',right_on = '配管ID')
#'To'データの削除
data12 = data12.drop("To",axis=1)
for col_name in data12.columns:
    print(col_name,data12[col_name].dtype)
#'配管ID'などの文字データのダミー化
data12_dummy = pd.get_dummies(data12[data12.columns[data12.columns != 'From']])
data12_dummy['From'] = data12['From']
#Support.csvの読み込み
data_out = pd.read_csv("drive/My Drive/mdb/test/C61/Support.csv",encoding='shift_jis')
#使用したい "Point","Type"データのみにする
for col_name in data_out.columns:
    if col_name != "Point" and col_name != "Type" :
        data_out = data_out.drop(col_name,axis=1)
#重複しているデータの削除
data_out = data_out.drop_duplicates()
#入力データと出力データを'Point'データで結合
data_press_type = pd.merge(data12_dummy,data_out,how = "outer",left_on = 'From',right_on='Point')
#'Support.csv'の'type'データの確認
data_series = Series(data_press_type['Type'])
pd.Series.value_counts(data_series)
#文字データなのでダミー化
data_press_type.at[data_press_type['Type'] == "Guide",'Type'] = 1
data_press_type.at[data_press_type['Type'] == "V - Stop",'Type'] = 2
data_press_type.at[data_press_type['Type'] == "Spr. Hgr",'Type'] = 3
data_press_type.at[data_press_type['Type'] == "Inclined",'Type'] = 4
data_test_press = data_press_type.fillna(0)
data_series = Series(data_test_press['Type'])
print(pd.Series.value_counts(data_series))
data_test_c61 = data_test_press[data_test_press["呼び径 (mm)_x"] != 0]
data_test_c61

#L41

#圧力_温度_配管ID.csvの読み込み
data1 = pd.read_csv("drive/My Drive/mdb/test/L41/圧力_温度_配管ID.csv", encoding='shift_jis')
#配管特性.csvの読み込み
data2 = pd.read_csv('drive/My Drive/mdb/test/L41/配管特性.csv',encoding='shift_jis')
#圧力_温度_配管ID.csvのデータの配管IDに合わせて配管特性.csvのデータを結合
data1_frame = DataFrame(data1)
data2_frame = DataFrame(data2)
data12 = pd.merge(data1_frame,data2_frame,how='left',left_on='配管 ID',right_on = '配管ID')
#'To'データの削除
data12 = data12.drop("To",axis=1)
for col_name in data12.columns:
    print(col_name,data12[col_name].dtype)
#'配管ID'などの文字データのダミー化
data12_dummy = pd.get_dummies(data12[data12.columns[data12.columns != 'From']])
data12_dummy['From'] = data12['From']
#Support.csvの読み込み
data_out = pd.read_csv("drive/My Drive/mdb/test/L41/Support.csv",encoding='shift_jis')
#使用したい "Point","Type"データのみにする
for col_name in data_out.columns:
    if col_name != "Point" and col_name != "Type" :
        data_out = data_out.drop(col_name,axis=1)
#重複しているデータの削除
data_out = data_out.drop_duplicates()
#入力データと出力データを'Point'データで結合
data_press_type = pd.merge(data12_dummy,data_out,how = "outer",left_on = 'From',right_on='Point')
#'Support.csv'の'type'データの確認
data_series = Series(data_press_type['Type'])
pd.Series.value_counts(data_series)
#文字データなのでダミー化
data_press_type.at[data_press_type['Type'] == "Guide",'Type'] = 1
data_press_type.at[data_press_type['Type'] == "V - Stop",'Type'] = 2
data_press_type.at[data_press_type['Type'] == "Spr. Hgr",'Type'] = 3
data_press_type.at[data_press_type['Type'] == "Inclined",'Type'] = 4
data_test_press = data_press_type.fillna(0)
data_series = Series(data_test_press['Type'])
print(pd.Series.value_counts(data_series))
data_test_l41 = data_test_press[data_test_press["呼び径 (mm)_x"] != 0]
data_test_l41

#P36

#圧力_温度_配管ID.csvの読み込み
data1 = pd.read_csv("drive/My Drive/mdb/test/P36/圧力_温度_配管ID.csv", encoding='shift_jis')
#配管特性.csvの読み込み
data2 = pd.read_csv('drive/My Drive/mdb/test/P36/配管特性.csv',encoding='shift_jis')
#圧力_温度_配管ID.csvのデータの配管IDに合わせて配管特性.csvのデータを結合
data1_frame = DataFrame(data1)
data2_frame = DataFrame(data2)
data12 = pd.merge(data1_frame,data2_frame,how='left',left_on='配管 ID',right_on = '配管ID')
#'To'データの削除
data12 = data12.drop("To",axis=1)
for col_name in data12.columns:
    print(col_name,data12[col_name].dtype)
#'配管ID'などの文字データのダミー化
data12_dummy = pd.get_dummies(data12[data12.columns[data12.columns != 'From']])
data12_dummy['From'] = data12['From']
#Support.csvの読み込み
data_out = pd.read_csv("drive/My Drive/mdb/test/P36/Support.csv",encoding='shift_jis')
#使用したい "Point","Type"データのみにする
for col_name in data_out.columns:
    if col_name != "Point" and col_name != "Type" :
        data_out = data_out.drop(col_name,axis=1)
#重複しているデータの削除
data_out = data_out.drop_duplicates()
#入力データと出力データを'Point'データで結合
data_press_type = pd.merge(data12_dummy,data_out,how = "outer",left_on = 'From',right_on='Point')
#'Support.csv'の'type'データの確認
data_series = Series(data_press_type['Type'])
pd.Series.value_counts(data_series)
#文字データなのでダミー化
data_press_type.at[data_press_type['Type'] == "Guide",'Type'] = 1
data_press_type.at[data_press_type['Type'] == "V - Stop",'Type'] = 2
data_press_type.at[data_press_type['Type'] == "Spr. Hgr",'Type'] = 3
data_press_type.at[data_press_type['Type'] == "Inclined",'Type'] = 4
data_test_press = data_press_type.fillna(0)
data_series = Series(data_test_press['Type'])
print(pd.Series.value_counts(data_series))
data_test_p36 = data_test_press[data_test_press["呼び径 (mm)_x"] != 0]
data_test_p36

#データの結合

data_test_c61 = pd.DataFrame(data_test_c61)
data_test_l41 = pd.DataFrame(data_test_l41)
data_test_p36 = pd.DataFrame(data_test_p36)
data_test_press = pd.concat([data_test_c61,data_test_l41,data_test_p36])
data_test_press = data_test_press.fillna(0)
data_test_press

From object
材質 object
配管 ID object
呼び径 (mm)_x float64
ｹｰｽ 1 圧力 (N/mm2) float64
ｹｰｽ 1 温度 (deg C) float64
ｹｰｽ 1 自動_1 float64
ｹｰｽ 1 線膨張量 (mm/m) float64
ｹｰｽ 1 自動_2 float64
ｹｰｽ 1 運転時弾性係数 (E6 N/mm2) float64
ｹｰｽ 1 自動_3 float64
ｹｰｽ 1 運転時許容応力 (N/mm2) float64
配管ID object
ﾀｸﾞ ﾅﾝﾊﾞｰ object
呼び径 (mm)_y int64
実外径 (mm) float64
ｽｹｼﾞｭｰﾙ int64
肉厚 (mm) float64
腐れ代 (mm) float64
製作公差 (mm) float64
保温厚さ (mm) float64
保温材質 object
保温材密度 (kg/m3) float64
外装板厚 (mm) float64
外装板材質 object
外装板密度 (kg/m3) object
ﾗｲﾆﾝｸﾞ厚さ (mm) float64
ﾗｲﾆﾝｸﾞ密度 (kg/m3) object
ﾗｲﾝ ｸﾗｽ object
流体比重 float64
配管材質 object
最低温度の警告 float64
溶接E係数 float64
溶接W係数 自動 object
溶接W係数 object
応力振幅 低減係数 float64
常温時許容応力 (N/mm2) float64
最小降伏点(Sy) (N/mm2) float64
引張強さ(Su) (N/mm2) float64
縦弾性係数 (E6 N/mm2) float64
周方向弾性係数 (E6 N/mm2) float64
せん断弾性係数 (E6 N/mm2) float64
密度 (kg/m3) float64
ﾎﾟｱｿﾝ比 float64
0    60
2     4
1     3
3     2
4     1
Name: Type, dtype: int64
From object
材質 object
配管 ID object
呼び径 (mm)_x float64
ｹｰｽ 1 圧力 (N/mm2) float64
ｹｰｽ 1 温度 (deg C) float6

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:126: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,From,Point,Type,せん断弾性係数 (E6 N/mm2),保温厚さ (mm),保温材密度 (kg/m3),保温材質_Other,周方向弾性係数 (E6 N/mm2),呼び径 (mm)_x,呼び径 (mm)_y,外装板厚 (mm),外装板密度 (kg/m3)_,外装板材質_,実外径 (mm),密度 (kg/m3),常温時許容応力 (N/mm2),引張強さ(Su) (N/mm2),応力振幅 低減係数,最低温度の警告,最小降伏点(Sy) (N/mm2),材質_STPA12-S,材質_STPG370-S,流体比重,溶接E係数,溶接W係数 自動_,溶接W係数_,縦弾性係数 (E6 N/mm2),肉厚 (mm),腐れ代 (mm),製作公差 (mm),配管 ID_E300S120,配管 ID_P100S40,配管 ID_P125S40,配管 ID_P150S40,配管 ID_P200S40,配管 ID_P250S120,配管 ID_P300S40,配管 ID_P350S120,配管 ID_P350S40,配管 ID_P400S40,配管 ID_P450S40,配管ID_E300S120,配管ID_P100S40,配管ID_P125S40,配管ID_P150S40,配管ID_P200S40,配管ID_P250S120,配管ID_P300S40,配管ID_P350S120,配管ID_P350S40,配管ID_P400S40,配管ID_P450S40,配管材質_STPA12-S,配管材質_STPG370-S,ｹｰｽ 1 圧力 (N/mm2),ｹｰｽ 1 温度 (deg C),ｹｰｽ 1 線膨張量 (mm/m),ｹｰｽ 1 自動_1,ｹｰｽ 1 自動_2,ｹｰｽ 1 自動_3,ｹｰｽ 1 運転時弾性係数 (E6 N/mm2),ｹｰｽ 1 運転時許容応力 (N/mm2),ｽｹｼﾞｭｰﾙ,ﾀｸﾞ ﾅﾝﾊﾞｰ_,ﾎﾟｱｿﾝ比,ﾗｲﾆﾝｸﾞ厚さ (mm),ﾗｲﾆﾝｸﾞ密度 (kg/m3)_,ﾗｲﾝ ｸﾗｽ_BB21,ﾗｲﾝ ｸﾗｽ_BB22,ﾗｲﾝ ｸﾗｽ_EH21
0,A00,0,0,0.07,150.0,201.23,1.0,0.19,300.0,300.0,0.0,1.0,1.0,318.5,7850.0,95.0,380.0,1.0,0.0,205.0,1.0,0.0,0.0,1.0,1.0,1.0,0.19,25.4,0.0,3.17,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,10.60,487.0,6.58,1.0,1.0,1.0,0.15,83.51,120.0,1.0,0.3,0.0,1.0,0.0,0.0,1.0
1,A01,0,0,0.07,150.0,201.23,1.0,0.19,350.0,350.0,0.0,1.0,1.0,355.6,7850.0,95.0,380.0,1.0,0.0,205.0,1.0,0.0,0.0,1.0,1.0,1.0,0.19,27.8,0.0,3.47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,10.60,487.0,6.58,1.0,1.0,1.0,0.15,83.51,120.0,1.0,0.3,0.0,1.0,0.0,0.0,1.0
2,A02 N,0,0,0.07,150.0,201.23,1.0,0.19,350.0,350.0,0.0,1.0,1.0,355.6,7850.0,95.0,380.0,1.0,0.0,205.0,1.0,0.0,0.0,1.0,1.0,1.0,0.19,27.8,0.0,3.47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,10.60,487.0,6.58,1.0,1.0,1.0,0.15,83.51,120.0,1.0,0.3,0.0,1.0,0.0,0.0,1.0
3,A02 F,0,0,0.07,150.0,201.23,1.0,0.19,350.0,350.0,0.0,1.0,1.0,355.6,7850.0,95.0,380.0,1.0,0.0,205.0,1.0,0.0,0.0,1.0,1.0,1.0,0.19,27.8,0.0,3.47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,10.60,487.0,6.58,1.0,1.0,1.0,0.15,83.51,120.0,1.0,0.3,0.0,1.0,0.0,0.0,1.0
4,A03 N,0,0,0.07,150.0,201.23,1.0,0.19,350.0,350.0,0.0,1.0,1.0,355.6,7850.0,95.0,380.0,1.0,0.0,205.0,1.0,0.0,0.0,1.0,1.0,1.0,0.19,27.8,0.0,3.47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,10.60,487.0,6.58,1.0,1.0,1.0,0.15,83.51,120.0,1.0,0.3,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,B45 N,0,0,0.07,30.0,201.22,1.0,0.20,125.0,125.0,0.0,1.0,1.0,139.8,7850.0,92.0,370.0,1.0,0.0,215.0,0.0,1.0,1.0,1.0,1.0,1.0,0.20,6.6,0.0,0.82,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.08,120.0,1.15,1.0,1.0,1.0,0.19,106.00,40.0,1.0,0.3,0.0,1.0,0.0,1.0,0.0
116,B45 F,0,0,0.07,30.0,201.22,1.0,0.20,125.0,125.0,0.0,1.0,1.0,139.8,7850.0,92.0,370.0,1.0,0.0,215.0,0.0,1.0,1.0,1.0,1.0,1.0,0.20,6.6,0.0,0.82,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.08,120.0,1.15,1.0,1.0,1.0,0.19,106.00,40.0,1.0,0.3,0.0,1.0,0.0,1.0,0.0
117,B46 N,0,0,0.07,30.0,201.22,1.0,0.20,125.0,125.0,0.0,1.0,1.0,139.8,7850.0,92.0,370.0,1.0,0.0,215.0,0.0,1.0,1.0,1.0,1.0,1.0,0.20,6.6,0.0,0.82,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.08,120.0,1.15,1.0,1.0,1.0,0.19,106.00,40.0,1.0,0.3,0.0,1.0,0.0,1.0,0.0
118,B46 F,0,0,0.07,30.0,201.22,1.0,0.20,125.0,125.0,0.0,1.0,1.0,139.8,7850.0,92.0,370.0,1.0,0.0,215.0,0.0,1.0,1.0,1.0,1.0,1.0,0.20,6.6,0.0,0.82,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.08,120.0,1.15,1.0,1.0,1.0,0.19,106.00,40.0,1.0,0.3,0.0,1.0,0.0,1.0,0.0


In [9]:
np.random.seed(0) # 乱数を固定値で初期化し再現性を持たせる
X = data_test_press.drop(['From','Point','Type'],axis=1)
T = data_test_press['Type']

X_train,X_test, y_train, y_test = train_test_split(X, T, train_size=0.7, test_size=0.3)

from sklearn.metrics import confusion_matrix,accuracy_score
 
#決定木
from sklearn.tree import DecisionTreeClassifier
 
dt = DecisionTreeClassifier(max_depth=3,random_state=0)
dt.fit(X_train,y_train)
y_pred = dt.predict(X_test)
print("dt_accuracy_score : {}".format(accuracy_score(y_test,y_pred)))
 
 
#ランダムフォレスト
from sklearn.ensemble import RandomForestClassifier
 
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
print("rf_accuracy_score : {}".format(accuracy_score(y_test,y_pred)))
 
#XGBoost
import xgboost as xgb
from xgboost import XGBClassifier
 
xgb = xgb.XGBClassifier(booster="gbtree",silent=0,seed=0)
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
print("xgb_accuracy_score : {}".format(accuracy_score(y_test,y_pred)))

#3つのモデルの特徴量重要度を並べてみる
models = ["DT","RF","XGB"]
importances = pd.DataFrame({"features":X_train.columns.values,
                            models[0]:dt.feature_importances_,
                            models[1]:rf.feature_importances_,
                            models[2]:xgb.feature_importances_})
 
#データフレームをDTの降順に並び替えて表示
importances.sort_values("DT",ascending=False)

dt_accuracy_score : 0.7888888888888889
rf_accuracy_score : 0.7888888888888889
xgb_accuracy_score : 0.7888888888888889


,features,DT,RF,XGB
51,ｹｰｽ 1 圧力 (N/mm2),0.654913,0.133498,0.428147
37,配管 ID_P450S40,0.159501,0.012976,0.000000
45,配管ID_P350S120,0.123865,0.014858,0.000000
10,実外径 (mm),0.061722,0.078251,0.000000
43,配管ID_P250S120,0.000000,0.004173,0.000000
48,配管ID_P450S40,0.000000,0.013738,0.000000
47,配管ID_P400S40,0.000000,0.020531,0.000000
46,配管ID_P350S40,0.000000,0.002504,0.000000
44,配管ID_P300S40,0.000000,0.028686,0.000000
42,配管ID_P200S40,0.000000,0.000184,0.000000
